# Contains Neural Network and IID Cross-Validation

In [15]:
#calculate Baseline Score!!!!!!!!!!!!!!!!!!!!!!!!!!!

In [16]:
import numpy as np
import pandas as pd
import copy

from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import PredefinedSplit

#just for testing:
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV

In [17]:
#reading dataframe
df = pd.read_csv("dataset/phase_3_TRAIN_7d499bff69ca69b6_6372c3e_MLPC2021_generic.csv")

In [18]:
# drop target value, student annotations and string ID from input features:
X = df.drop(columns=['quadrant','mean_A','mean_V','id','score_mode','score_key_strength'])

# we want to predict the quadrant:
y = df['quadrant'].values

In [19]:
#splitting by pianist and piecw!

#create tags_dataframe:
X_tags=pd.DataFrame()
X_tags['id']=df['id']


#extract piece_id and pianist to later allow by piece/pianist/both cross validation
def extractPianist(x):
    return x[0:2]
def extract_piece_id(x):
    return x[3:5]
def extract_snippet_number(x):
    return x[6:9]

X_tags['Pianist']=X_tags['id'].apply(extractPianist)
X_tags['Piece_id']=X_tags['id'].apply(extract_piece_id)
X_tags['Snippet_number']=X_tags['id'].apply(extract_snippet_number)
X_tags['class']=df['quadrant']

X_tags.head()

,id,Pianist,Piece_id,Snippet_number,class
0,GG-01-000,GG,01,000,1
1,GG-01-001,GG,01,001,1
2,GG-01-002,GG,01,002,1
3,GG-01-003,GG,01,003,1
4,GG-01-004,GG,01,004,1


In [20]:
#get list of pianists and pieces!
pianist_list=list(set(X_tags['Pianist']))
piece_list=list(set(X_tags['Piece_id']))
len(piece_list)

36

In [21]:
#get distribution of classes per pianist
pianist_dist_list=[]
pianist_dist_list_perecent=[]
for pianist in pianist_list:  
    mylist=[]
    for i in range(4):
        mylist.append(len(X_tags.loc[(X_tags['Pianist'] == pianist) & (X_tags['class']==i+1)]))
    pianist_dist_list.append(mylist)
    mylist=[element/sum(mylist) for element in mylist]
    pianist_dist_list_perecent.append(mylist)

In [22]:
pianist_dist_list

[[70, 46, 294, 72],
 [73, 48, 210, 95],
 [95, 62, 148, 65],
 [71, 58, 183, 70],
 [70, 28, 280, 83],
 [94, 32, 265, 95]]

In [23]:
#distribution of classes per pianist seems to be fairly uniform accross all pianist: ->reandom cv_split!

In [24]:
#choose random pianists
pianist_cv=[]
number_folds=3

#initialize pianists_cv:
for i in range(number_folds):
    pianist_cv.append([])
    
#assign pianists to folds randomly
cp_pianist_list=pianist_list.copy()
for i in range(len(cp_pianist_list)):
    choosen_pianist=cp_pianist_list[np.random.randint(len(cp_pianist_list), size=1)[0]]
    cp_pianist_list.remove(choosen_pianist)
    pianist_cv[len(cp_pianist_list)%number_folds].append(choosen_pianist)

In [25]:
pianist_cv

[['GG', 'FG'], ['RT', 'AS'], ['SR', 'AH']]

In [26]:
#get distribution of classes per piece
piece_dist_list=[]
piece_dist_list_perecent=[]
for piece in piece_list:  
    mylist=[]
    for i in range(4):
        mylist.append(len(X_tags.loc[(X_tags['Piece_id'] == piece) & (X_tags['class']==i+1)]))
    piece_dist_list.append(mylist)
    mylist=[element/sum(mylist) for element in mylist]
    piece_dist_list_perecent.append(mylist)
    

In [27]:
#piece_dist_list 
# shows that a lot of pieces are only labled as one emotion! To ensure that all 4 emotions a present
# in the test set we can not do a completly random split! IDEA: Create 4 Subesets such that in each subset the majority class of 
# every piece is the same and then assign sumples from each subset randomly and uiformly to the train test splits

In [30]:
#create 4 subsets:
piece_subsets=[]

#initialize piece_cv:
for i in range(4):
    piece_subsets.append([])
piece_list_cp=copy.deepcopy(piece_list)
for p,piece in enumerate(piece_list):
    done=False
    for i in range(4):
        if not done:
            #print(piece_dist_list[p][i])
            #print(max(piece_dist_list[p]))
            if piece_dist_list[p][i]==max(piece_dist_list[p][:]):
                piece_subsets[i].append(piece_list[p])
                piece_list_cp.remove
                done=True

In [31]:
#just double checking!
np.shape(piece_subsets)
piece_subsets
mysum=0
for i in range(4):
    mysum+=len(piece_subsets[i])
mysum

36

In [32]:
# chose random pieces using above subset technique:
piece_cv=[]
number_folds=5
#initialize piece_cv:
for i in range(number_folds):
    piece_cv.append([])
    
cp_pi_subs = copy.deepcopy(piece_subsets)
#assign pieces to folds randomly
for i in range(4):
    fold=0
    for p in range(len(piece_subsets[i])):
        chos_piece=cp_pi_subs[i][np.random.randint(len(cp_pi_subs[i]), size=1)[0]]
        cp_pi_subs[i].remove(chos_piece)
        piece_cv[fold].append(chos_piece)
        if fold>=number_folds-1:
            fold=0
        else:
            fold+=1


In [33]:
piece_cv

[['27', '41', '39', '04', '15', '35', '16', '46', '45'],
 ['37', '29', '11', '47', '08', '02', '34'],
 ['30', '21', '20', '32', '28', '25', '26'],
 ['05', '18', '03', '12', '24', '01', '22'],
 ['09', '14', '31', '36', '19', '38']]

In [34]:
#just double checking!
np.shape(piece_cv)
piece_subsets
mysum=0
for i in range(len(piece_cv)):
    mysum+=len(piece_cv[i])
mysum

36

In [ ]:
#  mask=np.where((X_tags['Pianist'] == pianist_cv[f][i]))[0].index.values.astype(int)

In [78]:
#creating masks for gridsearch cv:

#create vertical cv_splits
myCViterator = []
for f,fold in enumerate(pianist_cv):
    vetrical_mask_train=np.zeros(0)
    vetrical_mask_test=np.zeros(0)
    for i in range(len(fold)):
        mask= mask#X_tags[X_tags['Pianist']!=pianist_cv[f][i]].index.values.astype(int)
        vetrical_mask_train=np.concatenate((vetrical_mask_train,mask), axis=0)
        
        mask= X_tags[X_tags['Pianist']==pianist_cv[f][i]].index.values.astype(int)
        vetrical_mask_test=mask#np.concatenate((vetrical_mask_test,mask), axis=0)
    #print(vetrical_mask_test)
    trainIndices=vetrical_mask_train
    testIndices=vetrical_mask_test
    myCViterator.append((trainIndices, testIndices))

    
'''
for i in range(nFolds):
    trainIndices = myDf[ myDf['cvLabel']!=i ].index.values.astype(int)
    testIndices =  myDf[ myDf['cvLabel']==i ].index.values.astype(int)
    myCViterator.append( (trainIndices, testIndices) )
vetrical_mask=[[2,3,4],[3,4,54]]
'''
myCViterator

[(array([8.520e+02, 8.530e+02, 8.540e+02, 8.550e+02, 8.560e+02, 8.570e+02,
         8.580e+02, 8.590e+02, 8.600e+02, 8.610e+02, 8.620e+02, 8.630e+02,
         8.640e+02, 8.650e+02, 8.660e+02, 8.670e+02, 8.680e+02, 8.690e+02,
         8.700e+02, 8.710e+02, 8.720e+02, 8.730e+02, 8.740e+02, 8.750e+02,
         8.760e+02, 8.770e+02, 8.780e+02, 8.790e+02, 8.800e+02, 8.810e+02,
         8.820e+02, 8.830e+02, 8.840e+02, 8.850e+02, 8.860e+02, 8.870e+02,
         8.880e+02, 8.890e+02, 8.900e+02, 8.910e+02, 8.920e+02, 8.930e+02,
         8.940e+02, 8.950e+02, 8.960e+02, 8.970e+02, 8.980e+02, 8.990e+02,
         9.000e+02, 9.010e+02, 9.020e+02, 9.030e+02, 9.040e+02, 9.050e+02,
         9.060e+02, 9.070e+02, 9.080e+02, 9.090e+02, 9.100e+02, 9.110e+02,
         9.120e+02, 9.130e+02, 9.140e+02, 9.150e+02, 9.160e+02, 9.170e+02,
         9.180e+02, 9.190e+02, 9.200e+02, 9.210e+02, 9.220e+02, 9.230e+02,
         9.240e+02, 9.250e+02, 9.260e+02, 9.270e+02, 9.280e+02, 9.290e+02,
         9.300e+02, 9.310

In [75]:
gs_cv = GridSearchCV(MLPClassifier(hidden_layer_sizes=(4,40)), param_grid = {}, cv=myCViterator)
gs_cv.fit(pd.DataFrame(X), y)

ValueError: No valid specification of the columns. Only a scalar, list or slice of all integers or all strings, or boolean mask is allowed

In [61]:
gs_cv.cv_results_

{'mean_fit_time': array([0.03590298]),
 'std_fit_time': array([0.00498939]),
 'mean_score_time': array([0.00448489]),
 'std_score_time': array([0.00050163]),
 'params': [{}],
 'split0_test_score': array([1.]),
 'split1_test_score': array([1.]),
 'mean_test_score': array([1.]),
 'std_test_score': array([0.]),
 'rank_test_score': array([1])}

In [179]:
hidden_layers_structur=(4,40) #(4,40) seems to be the optimum

In [180]:
#by pianist cross validation
score=0
for pianist in pianist_list:
    mask1=X_tags['Pianist']!=pianist
    mask2=X_tags['Pianist']==pianist
    X_train, y_train=X[mask1],y[mask1]
    X_test, y_test=X[mask2],y[mask2]
    #print(X_train)
    clf = MLPClassifier(random_state=1, hidden_layer_sizes=(4,40),max_iter=400).fit(X_train, y_train)
    score+=clf.score(X_test, y_test)
    #print(clf.score(X_test, y_test))

In [32]:
pianist_cross_validation_score=score/len(pianist_list)
print(pianist_cross_validation_score)

0.5580338008076287


In [33]:
#by piece cross validation
#split dataset into train and test data
score=0
for piece in piece_list:
    mask1=X_tags['Piece_id']!=piece
    mask2=X_tags['Piece_id']==piece
    #print(len(mask1))
    X_train, y_train=X[mask1],y[mask1]
    X_test, y_test=X[mask2],y[mask2]
    clf = MLPClassifier(random_state=1, hidden_layer_sizes=(5,50), max_iter=300).fit(X_train, y_train)
    score+=clf.score(X_test, y_test)# Wrong!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    #xou need to do test size correction! e.g score+=score*length(test set)
    # and finally score /all test set sizes together
    
    #print(clf.score(X_test, y_test))

In [34]:
piece_cross_validation_score=score/len(piece_list)
print(piece_cross_validation_score)

0.3626491402617777


In [35]:
#by piece and pianist cross validation
#split dataset into train and test data
score=0
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
c=0
for i, piece in enumerate(piece_list):
    for j, pianist in enumerate(pianist_list):
        c+=1
        # we can not do ((X_tags['Piece_id'] ==piece) & (X_tags['Pianist']==pianist)) and !((X_tags['Piece_id'] ==piece) & (X_tags['Pianist']==pianist))
        # because the we would have unwanted correaltions between 
        #((X_tags['Piece_id'] ==piece) & (X_tags['Pianist']==pianist))
        #and ((X_tags['Piece_id'] !=piece) & (X_tags['Pianist']==pianist))
        mask2 = ((X_tags['Piece_id'] ==piece) & (X_tags['Pianist']==pianist))
        mask1 = ((X_tags['Piece_id'] !=piece) & (X_tags['Pianist']!=pianist))
                         
        X_train, y_train=X[mask1],y[mask1]
        X_test, y_test=X[mask2],y[mask2]
        #print(X_train)
        clf = MLPClassifier(random_state=1,hidden_layer_sizes=hidden_layers_structur, max_iter=300).fit(X_train, y_train)
        score+=clf.score(X_test, y_test)
        if c%10==0:
            print(f'percent done: {(c)/(len(pianist_list)*len(piece_list))*100}')
    


E:\Anaconda3\envs\AI\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:568: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


percent done: 4.62962962962963


E:\Anaconda3\envs\AI\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:568: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


percent done: 9.25925925925926
percent done: 13.88888888888889
percent done: 18.51851851851852
percent done: 23.14814814814815


E:\Anaconda3\envs\AI\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:568: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


...

percent done: 60.18518518518518


E:\Anaconda3\envs\AI\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:568: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


percent done: 64.81481481481481
percent done: 69.44444444444444
percent done: 74.07407407407408
percent done: 78.70370370370371
percent done: 83.33333333333334
percent done: 87.96296296296296
percent done: 92.5925925925926


E:\Anaconda3\envs\AI\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:568: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


percent done: 97.22222222222221


In [36]:
both_cross_validation_score=score/(len(pianist_list)*len(piece_list))
print(both_cross_validation_score)

0.3455958986626032


In [ ]:
# the score is slightly lower, but one must keep in mind,
# that in order to eliminate all correlations between test and training set
# we dropped (!pianist and piece) and (pianist and !piece) 
# in other words our training set size was quite a bit smaller!